In [ ]:
# Installing necessary libraries
!pip install tensorflow keras opencv-python


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

# Path to the ZIP file in your Drive
zip_path = '/content/drive/MyDrive/Colab Notebooks/Plant.zip'

# Path to extract files
extract_path = '/content/plant_dataset'

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check if the data is extracted properly
print("Extracted folders:", os.listdir(extract_path))


In [ ]:
# List the contents of the extracted folder
for folder in os.listdir(extract_path):
    folder_path = os.path.join(extract_path, folder)
    if os.path.isdir(folder_path):
        print(f"Folder: {folder}")
        print("Files:", os.listdir(folder_path))


In [ ]:
# Check contents of a specific folder (e.g., 'Apple___scab')
sample_folder_path = os.path.join(extract_path, 'data', 'Apple___scab')
print("Sample files:", os.listdir(sample_folder_path))


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Set the dataset path
extract_path = '/content/plant_dataset'
image_size = (128, 128)
batch_size = 32

# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'data'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory=os.path.join(extract_path, 'data'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load the MobileNetV2 model (pretrained on ImageNet)
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Combine into a model
model = Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Add early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
model.fit(
    train_generator,
    epochs=5,  # You can increase this after testing
    validation_data=validation_generator,
    callbacks=[early_stop]
)


In [ ]:
# Save entire model in HDF5 format
model.save('plant_disease_model.h5')
print("Model saved as plant_disease_model.h5")


In [ ]:
from google.colab import files
files.download('/content/plant_disease_model.h5')


In [ ]:
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=validation_generator,
    steps_per_epoch=100,
    validation_steps=20,
    callbacks=[early_stop]
)


In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import os

folder_path = '/content/plant_dataset/data/Tomato___healthy'
print("Files:", os.listdir(folder_path)[:5])  # print first 5 file names


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Use a valid image from the folder
img_path = '/content/plant_dataset/data/Tomato___healthy/96258.jpg'

# Load and preprocess the image
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict using the trained model
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Get class name
class_names = list(train_generator.class_indices.keys())
print("Predicted class:", class_names[predicted_class])


In [ ]:
!pip install streamlit


In [ ]:
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import streamlit as st
import numpy as np

# Load the trained model
model = load_model('plant_disease_model.h5')

# Define a function for prediction
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    return predicted_class

# Set up Streamlit interface
st.title('Plant Disease Prediction App')
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    # Save the uploaded image
    with open("uploaded_image.jpg", "wb") as f:
        f.write(uploaded_file.getbuffer())

    # Predict on the uploaded image
    result = predict_image("uploaded_image.jpg")
    st.image("uploaded_image.jpg", caption='Uploaded Image', use_column_width=True)
    st.write(f"Predicted Class: {result}")


In [ ]:
!pip install pyngrok


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load model
model = load_model('plant_disease_model.h5')

st.title('Plant Disease Detection')
st.write('Upload an image of the plant leaf to check for disease.')

uploaded_file = st.file_uploader("Choose a plant leaf image...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    img = image.load_img(uploaded_file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    prediction = model.predict(img_array)
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)

    if prediction[0] > 0.5:  # Example threshold; adjust as needed
        st.write("The plant is infected!")
    else:
        st.write("The plant is healthy!")


In [ ]:
!pip install streamlit pyngrok --quiet


In [ ]:
%%writefile app.py
import streamlit as st

st.title("🌿 Plant Disease Detection")
st.write("Upload a leaf image to check for disease (Demo)")


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained model
model = load_model("plant_disease_model.h5")

# Class labels (update based on your training data)
class_labels = [
    "Apple Scab", "Apple Black Rot", "Apple Cedar Rust", "Apple Healthy",
    "Corn Gray Leaf Spot", "Corn Common Rust", "Corn Northern Leaf Blight", "Corn Healthy",
    "Grape Black Rot", "Grape Esca", "Grape Leaf Blight", "Grape Healthy"
]

st.title("🌿 Plant Disease Detection")
st.write("Upload a leaf image to identify the disease.")

uploaded_file = st.file_uploader("Choose a leaf image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).resize((224, 224))
    st.image(image, caption="Uploaded Leaf Image", use_column_width=True)

    # Preprocess the image
    img_array = img_to_array(image)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]

    st.success(f"🔍 Predicted Disease: **{predicted_class}**")


In [ ]:
!ls

In [ ]:
!ngrok config add-authtoken 2x1qcQnqCnJheTfWi6ctlAoW0d2_sjjqG2WbEYiFxZ3NWBQd


In [ ]:
# (Optional) Kill old tunnels
from pyngrok import ngrok
ngrok.kill()

# Run Streamlit app in background
!streamlit run app.py --server.port 8501 &

# Start ngrok tunnel
public_url = ngrok.connect(addr=8501, proto="http")
print("🌐 Public URL:", public_url)
